In [ ]:
!pip install gspread

In [ ]:
!pip install gspread oauth2client

In [ ]:
from google.colab import auth
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()

# Create a connection to the Google Sheets API
gc = gspread.authorize(GoogleCredentials.get_application_default())

NameError: name 'GoogleCredentials' is not defined

In [11]:
import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io

# Authenticate and create the PyDrive client.
auth.authenticate_user()

# Build the Drive service
credentials = GoogleCredentials.get_application_default()
drive_service = build('drive', 'v3')

# Function to convert Google Sheets to Excel files
def convert_gsheets_to_xlsx(folder_path):
    # Search for .gsheet files in the given directory
    folder_path = folder_path.rstrip('/')
    files_in_directory = os.listdir(folder_path)

    for file_name in files_in_directory:
        # Only process files with ".gsheet" in the file name
        if file_name.endswith('.gsheet'):
            # Define the expected .xlsx file name
            xlsx_file_name = file_name.replace('.gsheet', '.xlsx')
            xlsx_file_path = os.path.join(folder_path, xlsx_file_name)

            # Check if the .xlsx file already exists
            if os.path.exists(xlsx_file_path):
                print(f'Mirrored file {xlsx_file_name} already exists. Skipping conversion for {file_name}.')
                continue

            # Get the file ID based on file name
            file_id = get_file_id(file_name)

            if file_id:
                # Download the file as .xlsx
                request = drive_service.files().export_media(
                    fileId=file_id,
                    mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                )
                file_bytes = io.BytesIO()
                downloader = MediaIoBaseDownload(file_bytes, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()

                # Save the downloaded .xlsx file
                with open(xlsx_file_path, 'wb') as f:
                    f.write(file_bytes.getvalue())
                print(f'Converted {file_name} to {xlsx_file_name}')
            else:
                print(f"File ID not found for {file_name}")

# Helper function to get the Google Drive file ID from a file name
def get_file_id(file_name):
    # Prepare the file name for the query
    base_name = file_name.replace('.gsheet', '')

    # Escape apostrophes for the query
    escaped_file_name = base_name.replace("'", "\\'")

    # Construct the query
    query = f"name = '{escaped_file_name}' and mimeType = 'application/vnd.google-apps.spreadsheet'"

    # Log the query for debugging
    print(f"Querying for: {query}")

    # List files in Google Drive and find the matching .gsheet file
    response = drive_service.files().list(
        q=query,
        spaces='drive'
    ).execute()

    files = response.get('files', [])
    if files:
        return files[0]['id']
    return None

# Run the conversion
folder_path = '/content/drive/MyDrive/Summer work/Sources for GDP Datasets'
convert_gsheets_to_xlsx(folder_path)

Querying for: name = 'Ards and North Down Gbr (1)' and mimeType = 'application/vnd.google-apps.spreadsheet'
File ID not found for Ards and North Down Gbr (1).gsheet
Mirrored file Ards and North Down Gbr.xlsx already exists. Skipping conversion for Ards and North Down Gbr.gsheet.
Mirrored file Western Macedonia GRC.xlsx already exists. Skipping conversion for Western Macedonia GRC.gsheet.
Mirrored file TLC22	Tyneside GBR.xlsx already exists. Skipping conversion for TLC22	Tyneside GBR.gsheet.
Querying for: name = 'Untitled spreadsheet (4)' and mimeType = 'application/vnd.google-apps.spreadsheet'
File ID not found for Untitled spreadsheet (4).gsheet
Querying for: name = 'Untitled spreadsheet (3)' and mimeType = 'application/vnd.google-apps.spreadsheet'
File ID not found for Untitled spreadsheet (3).gsheet
Querying for: name = 'Untitled spreadsheet (2)' and mimeType = 'application/vnd.google-apps.spreadsheet'
File ID not found for Untitled spreadsheet (2).gsheet
Querying for: name = 'Untit